# DGA Detection, Modeling and Learning


## Load Data

In [23]:
import pandas as pd
import os
import torch
import torch.utils.data

In [24]:
# !ls -la data

In [25]:
# should be the name of directory you created to save your features data
data_dir = 'data'

In [26]:
# take a look at some matsnu example domains
from dga import matsnu

for i in range(10):
    print(matsnu.generate_domain())

commentprintreasonneckdare.com
kingrepresentfirmcommunicate.com
groundtranslatechallenge.com
midnightclaimsourcereduce.com
clouddisappointedentrance.com
wheelpositionboxkneedeal.com
dotexercisebidboneimprove.com
clickshockdumpamazingcomment.com
toprevealpressreplymarried.com
platformruinsmokesmartbone.com


In [27]:
matsnu_list = []

for i in range(10000):
    matsnu_list.append(matsnu.generate_domain())
    
matsnu_df = pd.DataFrame(matsnu_list, columns=['domain'])

print("Matsnu Shape:", matsnu_df.shape)

matsnu_df.head()

Matsnu Shape: (10000, 1)


,domain
0,chocolateneckcarpetcarereceive.com
1,financecarrychartreducediscover.com
2,freedomnailrewardholdhang.com
3,boardpleasureoccasionraise.com
4,writerdoesclubrubprintpriest.com


In [22]:
# save in data file
matsnu_df.to_csv(data_dir + "/matsnu.csv")

In [ ]:
# alex top 1 million
read_csv(data_dir + "/")

---

# Modeling

---


In [1]:
# Directory of train.py
!pygmentize model/train.py

import subprocess as sb 
import sys 

sb.call([sys.executable, "-m", "pip", "install", 'pandas']) 

import argparse
import json
import os
import pandas as pd
import torch
import torch.optim as optim
import torch.utils.data

# imports the model in model.py by name
from model import BinaryClassifier


def model_fn(model_dir):
    """Load the PyTorch model from the `model_dir` directory."""
    print("Loading model.")

    # First, load the parameters used to create the model.
    model_info = {}
    model_info_path = os.path.join(model_dir, 'model_info.pth')
    with open(model_info_path, 'rb') as f:
        model_info = torch.load(f)

    print("model_info: {}".format(model_info))

    # Determine the device and construct the model.
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = BinaryClassifier(model_info['input_features'], model_info['hidden_dim'], model_info['output_dim'])

    # Load the stored model parameters.
    model_path = os.path.join(mo

In [ ]:
# torch imports
import torch
import torch.nn.functional as F
import torch.nn as nn

class LSTMClassifier(nn.Module):
    """
    This is the simple RNN model we will be using to perform Sentiment Analysis.
    """

    def __init__(self, embedding_dim, hidden_dim, vocab_size):
        """
        Initialize the model by settingg up the various layers.
        """
        super(LSTMClassifier, self).__init__()

        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim)
        self.dense = nn.Linear(in_features=hidden_dim, out_features=1)
        self.sig = nn.Sigmoid()
        
        self.word_dict = None

    def forward(self, x):
        """
        Perform a forward pass of our model on some input.
        """
        x = x.t()
        lengths = x[0,:]
        reviews = x[1:,:]
        embeds = self.embedding(reviews)
        lstm_out, _ = self.lstm(embeds)
        out = self.dense(lstm_out)
        out = out[lengths - 1, range(len(lengths))]
        return self.sig(out.squeeze())

---
# Create an Estimator



## Define PyTorch estimators

In [2]:
train_data = pd.read_csv(os.path.join(data_dir, "train.csv"), header=None, names=None)

train_y = torch.from_numpy(train_data[[0]].values).float().squeeze()
train_x = torch.from_numpy(train_data.drop([0], axis=1).values).float()

train_ds = torch.utils.data.TensorDataset(train_x, train_y)

train_sample_dl = torch.utils.data.DataLoader(train_ds, batch_size=10)

NameError: name 'data_dir' is not defined

In [ ]:
# Training function for LSTM
def train_lstm(model, train_loader, epochs, criterion, optimizer, device):
    """
    This is the training method that is called by the PyTorch training script of the LSTM model. The parameters
    passed are as follows:
    model        - The PyTorch model that we wish to train.
    train_loader - The PyTorch DataLoader that should be used during training.
    epochs       - The total number of epochs to train for.
    criterion    - The loss function used for training. 
    optimizer    - The optimizer to use during training.
    device       - Where the model and data should be loaded (gpu or cpu).
    """
    
    # training loop is provided
    for epoch in range(1, epochs + 1):
        
        model.train() # Make sure that the model is in training mode.

        total_loss = 0

        for batch in train_loader:
            
            # get data
            batch_x, batch_y = batch
            
            # 
            batch_x = torch.from_numpy(batch_x).float().squeeze()
            batch_y = torch.from_numpy(batch_y).float()

            batch_x = batch_x.to(device)
            batch_y = batch_y.to(device)

            optimizer.zero_grad()
            
            model.hidden_cell = (torch.zeros(1, 1, model.hidden_layer_dim),
                torch.zeros(1, 1, model.hidden_layer_dim))

            # get predictions from model
            y_pred = model(batch_x)
            
            # perform backprop
            loss = criterion(y_pred, batch_y)
            loss.backward()
            optimizer.step()
            
            total_loss += loss.data.item()
            
        if epoch%25 == 1:
            print("Epoch: {}, Loss: {}".format(epoch, total_loss / len(train_loader)))

In [3]:
import torch.optim as optim
# from model.LSTM_Estimator import LSTMEstimator

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = LSTMEstimator(8, 30, 1, 8)

optimizer = optim.Adam(model.parameters(), lr=0.001)

loss_fn = torch.nn.L1Loss()

train_lstm(model, processed_data, 100, loss_fn, optimizer, device)

ModuleNotFoundError: No module named 'source_pytorch'